In [1]:
# Importando a CNN pré-treinada
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
model = ResNet50(weights="imagenet", include_top=False, pooling=max)


94781440/94765736 [==============================] - 1s 0us/step


In [2]:
# carregando a base de dados
from keras.datasets import cifar10
(trainX, trainY), (testX, testY) = cifar10.load_data()
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

170508288/170498071 [==============================] - 9s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [3]:
import cv2
import numpy as np
w=64
h=64
Xtrain_new=[]
#ytrain_new=[]

# aumentando o tamanho das imagens de treinamento
for i in range(trainX.shape[0]):
  im=cv2.resize(trainX[i], (w, h))
  Xtrain_new.append(im)
#  ytrain_new.append(trainY[i])

# aumentando o tamanho das imagens de teste
Xtest_new=[]
for i in range(testX.shape[0]):
  im=cv2.resize(testX[i], (w, h))
  Xtest_new.append(im)

# transformando lista em array
trainX=np.asarray(Xtrain_new)
testX=np.asarray(Xtest_new)
#trainY=np.asarray(ytrain_new)

del(Xtrain_new)
del(Xtest_new)
#del(ytrain_new)

print(trainX.shape)
print(testX.shape)

(50000, 64, 64, 3)
(10000, 64, 64, 3)


In [4]:
# Extração de características usando a CNN
trainX=preprocess_input(trainX)
testX=preprocess_input(testX)

# extraindo features das imagens de treinamento
feat_train=model.predict(trainX)
del(trainX)

# extraindo features das imagens de teste
feat_test=model.predict(testX)
del(testX)

print(feat_train.shape)
print(feat_test.shape)

(50000, 2, 2, 2048)
(10000, 2, 2, 2048)


In [5]:
# Reshape das fearures para transformar em um vetor (2x2x2048=8192)

feat_train=np.reshape(feat_train, (feat_train.shape[0], 8192))
feat_test=np.reshape(feat_test, (feat_test.shape[0], 8192))
print(feat_train.shape)
print(feat_test.shape)

(50000, 8192)
(10000, 8192)


In [6]:
# Criando um classificador com as features extraídas
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

y_train=np.ravel(trainY)
y_test=np.ravel(testY)

# declarando o indutor
clf=KNeighborsClassifier(n_neighbors=3)

# treinar o classificador
clf.fit(feat_train, y_train)

# avaliando o modelo
score=clf.score(feat_test, y_test)
print("Accuracy = %.2f" % (score*100))

# predição do modelo para cada exemplo de teste
y_pred=clf.predict(feat_test)

# matriz de confusão
ma=confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(ma)




Accuracy = 69.67
Matriz de Confusão:
[[811  23  32   4  19   2  10   6  64  29]
 [ 36 829   5   5  10   0   3   2  16  94]
 [ 84   9 663  24  79  17 108   7   5   4]
 [ 69  43  93 439  93  84 142  23   4  10]
 [ 49   8  84  22 622  10 137  59   5   4]
 [ 29  22  63 162  64 562  58  35   3   2]
 [ 20   7  59  36  48   2 824   1   2   1]
 [ 40  15  45  39 112  29   8 695   9   8]
 [110  46   8   8  15   1   8   1 775  28]
 [ 54 137   5   8   9   1   5   5  29 747]]
